In [66]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import math
from sklearn.naive_bayes import GaussianNB

Let's load the data

In [2]:
spi_rank_df = pd.read_csv('../data/spi_global_rankings.csv')
intl_spi_rank_df = pd.read_csv('../data/spi_global_rankings_intl.csv')
spi_match_df = pd.read_csv('../data/spi_matches.csv')
latest_spi_match_df = pd.read_csv('../data/spi_matches_latest.csv')

Let's do some EDA

In [3]:
spi_match_df

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2016,2016-07-09,7921,FA Women's Super League,Liverpool Women,Reading,51.56,50.42,0.4389,0.2767,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,2016-07-10,7921,FA Women's Super League,Arsenal Women,Notts County Ladies,46.61,54.03,0.3572,0.3608,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,2016-07-10,7921,FA Women's Super League,Chelsea FC Women,Birmingham City,59.85,54.64,0.4799,0.2487,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,2016-07-16,7921,FA Women's Super League,Liverpool Women,Notts County Ladies,53.00,52.35,0.4289,0.2699,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,2016-07-17,7921,FA Women's Super League,Chelsea FC Women,Arsenal Women,59.43,60.99,0.4124,0.3157,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42168,2020,2021-05-30,1871,Spanish Segunda Division,Mirandes,CD Sabadell,32.09,30.30,0.4473,0.2862,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42169,2020,2021-05-30,1871,Spanish Segunda Division,Rayo Vallecano,Lugo,39.38,29.25,0.5330,0.2140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42170,2020,2021-05-30,1871,Spanish Segunda Division,Albacete,Fuenlabrada,28.31,36.32,0.3328,0.3567,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42171,2020,2021-05-30,1871,Spanish Segunda Division,Tenerife,Real Oviedo,33.38,31.65,0.4338,0.2707,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


for right now let's just do the naive approach with just the only data frame that has a bunch of data which would be the match history with the spi of each team.

Ideas: 
• Split up the date, that could have an effect on game play

• Day of the week could affect game play

• Have classifier try and determine whether it will be a win/lose/tie and have that be a feature

• Use the features given, making sure to scale and normalize them

• I'm assuming that the team first is home or something like that. Have a bool that just predicts either the right or the left, essentially giving us 2 times the amount of data to work with

• make a dict between team name and an int. This is slightly more difficult given that there are two columns with team names in both columns

• Integrate the rank of the team (seems difficult just because the strings of team names don't match exactly)

• Get data of players from that time and match up with the team


In [76]:
limited_df = spi_match_df[['season', 'date', 'league_id', 'league', 'team1', 'team2', 'spi1', 'spi2', 'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2', 'score1', 'score2']]
limited_df[["year", "month", "day"]] = limited_df.date.str.split("-", expand=True)
limited_df = limited_df.astype({"year":'int32', "month":"int32", "day":"int32"})
limited_df.dropna(inplace=True)
limited_df["day_of_week"] = limited_df.apply(lambda row: datetime(row.year, row.month, row.day).weekday(), axis=1)

def win_los(row):
    if row.score1 == row.score2:
        return 1
    elif row.score1 > row.score2:
        return 0
    elif row.score1 < row.score2:
        return 2
    
limited_df["is_tie"] = limited_df.apply(lambda row: win_los(row), axis=1)

limited_df

/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,probtie,proj_score1,proj_score2,score1,score2,year,month,day,day_of_week,is_tie
0,2016,2016-07-09,7921,FA Women's Super League,Liverpool Women,Reading,51.56,50.42,0.4389,0.2767,0.2844,1.39,1.05,2.0,0.0,2016,7,9,5,0
1,2016,2016-07-10,7921,FA Women's Super League,Arsenal Women,Notts County Ladies,46.61,54.03,0.3572,0.3608,0.2819,1.27,1.28,2.0,0.0,2016,7,10,6,0
2,2016,2016-07-10,7921,FA Women's Super League,Chelsea FC Women,Birmingham City,59.85,54.64,0.4799,0.2487,0.2714,1.53,1.03,1.0,1.0,2016,7,10,6,1
3,2016,2016-07-16,7921,FA Women's Super League,Liverpool Women,Notts County Ladies,53.00,52.35,0.4289,0.2699,0.3013,1.27,0.94,0.0,0.0,2016,7,16,5,1
4,2016,2016-07-17,7921,FA Women's Super League,Chelsea FC Women,Arsenal Women,59.43,60.99,0.4124,0.3157,0.2719,1.45,1.24,1.0,2.0,2016,7,17,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34819,2020,2020-09-28,1854,Italy Serie A,Bologna,Parma,58.86,56.23,0.4403,0.2952,0.2646,1.56,1.23,4.0,1.0,2020,9,28,0,0
34820,2020,2020-09-28,1832,Belgian Jupiler League,Genk,KV Oostende,50.75,32.42,0.6421,0.1535,0.2044,2.11,0.92,2.0,2.0,2020,9,28,0,1
34821,2020,2020-09-28,1864,Portuguese Liga,Belenenses,Famalicao,40.18,43.84,0.4134,0.3133,0.2733,1.31,1.11,1.0,2.0,2020,9,28,0,2
34822,2020,2020-09-28,2411,Barclays Premier League,Liverpool,Arsenal,91.38,78.83,0.6600,0.1468,0.1932,2.25,0.96,3.0,1.0,2020,9,28,0,0


In [77]:
limited_df.drop("win_lose_draw", inplace=True, axis=1)

KeyError: "['win_lose_draw'] not found in axis"

In [78]:
#produce the dict of team to int
ids_teams = []
for row in tqdm(limited_df.itertuples()):
    if row.team1 not in ids_teams:
        ids_teams.append(row.team1)
    if row.team2 not in ids_teams:
        ids_teams.append(row.team2)

teams_ids = dict((name, i) for i, name in enumerate(ids_teams))
league_ids = pd.Series(limited_df.league_id.values, index=limited_df.league).to_dict()
ids_league = dict((league_ids[x], x) for x in league_ids)
limited_df

34237it [00:00, 69414.72it/s]


,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,probtie,proj_score1,proj_score2,score1,score2,year,month,day,day_of_week,is_tie
0,2016,2016-07-09,7921,FA Women's Super League,Liverpool Women,Reading,51.56,50.42,0.4389,0.2767,0.2844,1.39,1.05,2.0,0.0,2016,7,9,5,0
1,2016,2016-07-10,7921,FA Women's Super League,Arsenal Women,Notts County Ladies,46.61,54.03,0.3572,0.3608,0.2819,1.27,1.28,2.0,0.0,2016,7,10,6,0
2,2016,2016-07-10,7921,FA Women's Super League,Chelsea FC Women,Birmingham City,59.85,54.64,0.4799,0.2487,0.2714,1.53,1.03,1.0,1.0,2016,7,10,6,1
3,2016,2016-07-16,7921,FA Women's Super League,Liverpool Women,Notts County Ladies,53.00,52.35,0.4289,0.2699,0.3013,1.27,0.94,0.0,0.0,2016,7,16,5,1
4,2016,2016-07-17,7921,FA Women's Super League,Chelsea FC Women,Arsenal Women,59.43,60.99,0.4124,0.3157,0.2719,1.45,1.24,1.0,2.0,2016,7,17,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34819,2020,2020-09-28,1854,Italy Serie A,Bologna,Parma,58.86,56.23,0.4403,0.2952,0.2646,1.56,1.23,4.0,1.0,2020,9,28,0,0
34820,2020,2020-09-28,1832,Belgian Jupiler League,Genk,KV Oostende,50.75,32.42,0.6421,0.1535,0.2044,2.11,0.92,2.0,2.0,2020,9,28,0,1
34821,2020,2020-09-28,1864,Portuguese Liga,Belenenses,Famalicao,40.18,43.84,0.4134,0.3133,0.2733,1.31,1.11,1.0,2.0,2020,9,28,0,2
34822,2020,2020-09-28,2411,Barclays Premier League,Liverpool,Arsenal,91.38,78.83,0.6600,0.1468,0.1932,2.25,0.96,3.0,1.0,2020,9,28,0,0


In [79]:
final_df = limited_df
final_df.drop(["league", "date"], axis=1, inplace=True)

In [80]:
#replace words with numbers
final_df["team1_id"] = 0

import time
def set_team_id(row, num):
    if num == 1:
        return teams_ids[row.team1]
    elif num == 2:
        return teams_ids[row.team2]
    else:
        raise Exception("input is either 1 or 2 for second param")

final_df["team1_id"] = final_df.apply(lambda row: set_team_id(row, 1), axis=1)
final_df["team2_id"] = final_df.apply(lambda row: set_team_id(row, 2), axis=1)
all_num_df = final_df.drop(["team1", "team2"], axis=1)
all_num_df[["which_team", "predict"]] = 0
all_num_df

,season,league_id,spi1,spi2,prob1,prob2,probtie,proj_score1,proj_score2,score1,score2,year,month,day,day_of_week,is_tie,team1_id,team2_id,which_team,predict
0,2016,7921,51.56,50.42,0.4389,0.2767,0.2844,1.39,1.05,2.0,0.0,2016,7,9,5,0,0,1,0,0
1,2016,7921,46.61,54.03,0.3572,0.3608,0.2819,1.27,1.28,2.0,0.0,2016,7,10,6,0,2,3,0,0
2,2016,7921,59.85,54.64,0.4799,0.2487,0.2714,1.53,1.03,1.0,1.0,2016,7,10,6,1,4,5,0,0
3,2016,7921,53.00,52.35,0.4289,0.2699,0.3013,1.27,0.94,0.0,0.0,2016,7,16,5,1,0,3,0,0
4,2016,7921,59.43,60.99,0.4124,0.3157,0.2719,1.45,1.24,1.0,2.0,2016,7,17,6,2,4,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34819,2020,1854,58.86,56.23,0.4403,0.2952,0.2646,1.56,1.23,4.0,1.0,2020,9,28,0,0,79,403,0,0
34820,2020,1832,50.75,32.42,0.6421,0.1535,0.2044,2.11,0.92,2.0,2.0,2020,9,28,0,1,554,547,0,0
34821,2020,1864,40.18,43.84,0.4134,0.3133,0.2733,1.31,1.11,1.0,2.0,2020,9,28,0,2,332,745,0,0
34822,2020,2411,91.38,78.83,0.6600,0.1468,0.1932,2.25,0.96,3.0,1.0,2020,9,28,0,0,40,39,0,0


In [81]:
team1_df = all_num_df.drop(['score1', "score2"], axis=1)
team2_df = all_num_df.drop(["score1", "score2"], axis=1)
team2_df["which_team"] = 1
team1_df["predict"] = all_num_df["score1"]
team2_df["predict"] = all_num_df["score2"]
very_final_df = pd.concat([team1_df, team2_df])
very_final_df

,season,league_id,spi1,spi2,prob1,prob2,probtie,proj_score1,proj_score2,year,month,day,day_of_week,is_tie,team1_id,team2_id,which_team,predict
0,2016,7921,51.56,50.42,0.4389,0.2767,0.2844,1.39,1.05,2016,7,9,5,0,0,1,0,2.0
1,2016,7921,46.61,54.03,0.3572,0.3608,0.2819,1.27,1.28,2016,7,10,6,0,2,3,0,2.0
2,2016,7921,59.85,54.64,0.4799,0.2487,0.2714,1.53,1.03,2016,7,10,6,1,4,5,0,1.0
3,2016,7921,53.00,52.35,0.4289,0.2699,0.3013,1.27,0.94,2016,7,16,5,1,0,3,0,0.0
4,2016,7921,59.43,60.99,0.4124,0.3157,0.2719,1.45,1.24,2016,7,17,6,2,4,2,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34819,2020,1854,58.86,56.23,0.4403,0.2952,0.2646,1.56,1.23,2020,9,28,0,0,79,403,1,1.0
34820,2020,1832,50.75,32.42,0.6421,0.1535,0.2044,2.11,0.92,2020,9,28,0,1,554,547,1,2.0
34821,2020,1864,40.18,43.84,0.4134,0.3133,0.2733,1.31,1.11,2020,9,28,0,2,332,745,1,2.0
34822,2020,2411,91.38,78.83,0.6600,0.1468,0.1932,2.25,0.96,2020,9,28,0,0,40,39,1,1.0


Here we are going to add a classifier to add another feature because before adding the classifier we had an accuracy of around 10%. We can get that better. We want to use the same seperation of data so that there is no contamination of trying to perform regression on data that the classifer has already seen

In [117]:
#woo we are done! Let's split up the data
X_df = very_final_df.drop(["predict", "is_tie"], axis=1)
y_df = very_final_df[["predict", "is_tie"]]

X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X_df, y_df, test_size=0.2)

In [118]:
classify_X_train = np.array(X_train_df.drop(["which_team"], axis=1))
classify_y_train = np.array(y_train_df["is_tie"])
classify_X_test = np.array(X_test_df.drop(["which_team"], axis=1))
classify_y_test = np.array(y_test_df["is_tie"])

What different classifiers can we use?

• Decision Tree

• Naive Bayes

• Linear Regression

• Random Forrest

In [85]:
#Naive Bayes
classify_X_train
gauss = GaussianNB()
gauss.fit(classify_X_train, classify_y_train.astype('int'))
gauss.score(classify_X_test, classify_y_test)

0.470609711573567

In [87]:
#Decision Tree
dec_tree = DecisionTreeClassifier()
dec_tree.fit(classify_X_train, classify_y_train)
dec_tree.score(classify_X_test, classify_y_test)

0.8818546914932457

In [88]:
#Linear Regression
line = LinearRegression()
line.fit(classify_X_train, classify_y_train)
line.score(classify_X_test, classify_y_test)

0.12285641105006362

In [98]:
#Random Forrest
rand_forest = RandomForestRegressor(n_estimators=100, n_jobs=-1)
rand_forest.fit(classify_X_train, classify_y_train)
rand_forest.score(classify_X_test, classify_y_test)

0.7108548706410458

In [119]:
#insert the observation
import time
def dec_predict(x):
    final = []
    for i in x.keys():
        if i == "which_team":
            continue
        final.append(x[i])
    y = dec_tree.predict([np.array(final)])
    t = y[0]
    return t
X_df["pred_winner"] = X_df.apply(lambda x: dec_predict(x), axis=1)
X_df

,season,league_id,spi1,spi2,prob1,prob2,probtie,proj_score1,proj_score2,year,month,day,day_of_week,team1_id,team2_id,which_team,pred_winner
0,2016,7921,51.56,50.42,0.4389,0.2767,0.2844,1.39,1.05,2016,7,9,5,0,1,0,0
1,2016,7921,46.61,54.03,0.3572,0.3608,0.2819,1.27,1.28,2016,7,10,6,2,3,0,0
2,2016,7921,59.85,54.64,0.4799,0.2487,0.2714,1.53,1.03,2016,7,10,6,4,5,0,1
3,2016,7921,53.00,52.35,0.4289,0.2699,0.3013,1.27,0.94,2016,7,16,5,0,3,0,1
4,2016,7921,59.43,60.99,0.4124,0.3157,0.2719,1.45,1.24,2016,7,17,6,4,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34819,2020,1854,58.86,56.23,0.4403,0.2952,0.2646,1.56,1.23,2020,9,28,0,79,403,1,0
34820,2020,1832,50.75,32.42,0.6421,0.1535,0.2044,2.11,0.92,2020,9,28,0,554,547,1,1
34821,2020,1864,40.18,43.84,0.4134,0.3133,0.2733,1.31,1.11,2020,9,28,0,332,745,1,2
34822,2020,2411,91.38,78.83,0.6600,0.1468,0.1932,2.25,0.96,2020,9,28,0,40,39,1,0


In [126]:
X = np.array(X_df)
y = np.array(y_df.drop("is_tie", axis=1))

#for some reason not scaling makes it perform better?
# X = MinMaxScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Let's load up some regressors. But which to choose from? We can start with these
• Linear Regression

• Random Forrest Regressor

• NN?

In [127]:
linear = LinearRegression()
linear.fit(X_train, y_train)

linear.score(X_test, y_test)

0.04497172735975519

In [128]:
forest = RandomForestRegressor(n_estimators=200, n_jobs=-1)
forest.fit(X_train, y_train)

forest.score(X_test, y_test)

<ipython-input-128-cd91b9403101>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest.fit(X_train, y_train)


0.40094319561511027